In [1]:
!pip install feedparser

Processing c:\users\lenovo\appdata\local\pip\cache\wheels\8c\69\b7\f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f\feedparser-5.2.1-cp37-none-any.whl


In [2]:
!pip install dateparser

  Using cached regex-2020.6.8-cp37-cp37m-win_amd64.whl (268 kB)


In [3]:
import feedparser
from bs4 import BeautifulSoup
import urllib
from dateparser import parse as parse_date
import requests



class NEWS:
    def __init__(self, lang = 'en', country = 'US'):
        self.lang = lang.lower()
        self.country = country.upper()
        self.BASE_URL = 'https://news.google.com/rss'

    def __top_news_parser(self, text):
        """Return subarticles from the main and topic feeds"""
        try:
            bs4_html = BeautifulSoup(text, "html.parser")
            # find all li tags
            lis = bs4_html.find_all('li')
            sub_articles = []
            for li in lis:
                try:
                    sub_articles.append({"url": li.a['href'],
                                         "title": li.a.text,
                                         "publisher": li.font.text})
                except:
                    pass
            return sub_articles
        except:
            return text

    def __ceid(self):
        """Compile correct country-lang parameters for Google News RSS URL"""
        return '?ceid={}:{}&hl={}&gl={}'.format(self.country,self.lang,self.lang,self.country)

    def __add_sub_articles(self, entries):
        for i, val in enumerate(entries):
            if 'summary' in entries[i].keys():
                entries[i]['sub_articles'] = self.__top_news_parser(entries[i]['summary'])
            else:
                entries[i]['sub_articles'] = None
        return entries

    def __scaping_bee_request(self, api_key, url):
        response = requests.get(
            url="https://app.scrapingbee.com/api/v1/",
            params={
                "api_key": api_key,
                "url": url,
                "render_js": "false"
            }
        )
        if response.status_code == 200:
            return response
        if response.status_code != 200:
            raise Exception("ScrapingBee status_code: "  + str(response.status_code) + " " + response.text)

    def __parse_feed(self, feed_url, proxies=None, scraping_bee = None):

        if scraping_bee and proxies:
            raise Exception("Pick either ScrapingBee or proxies. Not both!")

        if proxies:
            r = requests.get(feed_url, proxies = proxies)
        else:
            r = requests.get(feed_url)

        if scraping_bee:
            r = self.__scaping_bee_request(url = feed_url, api_key = scraping_bee)
        else:
            r = requests.get(feed_url)


        if 'https://news.google.com/rss/unsupported' in r.url:
            raise Exception('This feed is not available')

        d = feedparser.parse(r.text)
        return dict((k, d[k]) for k in ('feed', 'entries'))

    def __search_helper(self, query):
        return urllib.parse.quote_plus(query)

    def __from_to_helper(self, validate=None):
        try:
            validate = parse_date(validate).strftime('%Y-%m-%d')
            return str(validate)
        except:
            raise Exception('Could not parse your date')



    def top_news(self, proxies=None, scraping_bee = None):
        """Return a list of all articles from the main page of Google News
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + self.__ceid(), proxies=proxies, scraping_bee=scraping_bee)
        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def topic_headlines(self, topic: str, proxies=None, scraping_bee=None):
        """Return a list of all articles from the topic page of Google News
        given a country and a language"""
        #topic = topic.upper()
        if topic.upper() in ['WORLD', 'NATION', 'BUSINESS', 'TECHNOLOGY', 'ENTERTAINMENT', 'SCIENCE', 'SPORTS', 'HEALTH']:
            d = self.__parse_feed(self.BASE_URL + '/headlines/section/topic/{}'.format(topic.upper()) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        else:
            d = self.__parse_feed(self.BASE_URL + '/topics/{}'.format(topic) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        if len(d['entries']) > 0:
            return d
        else:
            raise Exception('unsupported topic')

    def geo_headlines(self, geo: str, proxies=None, scraping_bee=None):
        """Return a list of all articles about a specific geolocation
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + '/headlines/section/geo/{}'.format(geo) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def search(self, query: str, helper = True, when = None, from_ = None, to_ = None, proxies=None, scraping_bee=None):
        """
        Return a list of all articles given a full-text search parameter,
        a country and a language
        :param bool helper: When True helps with URL quoting
        :param str when: Sets a time range for the artiles that can be found
        """

        if when:
            query += ' when:' + when

        if from_ and not when:
            from_ = self.__from_to_helper(validate=from_)
            query += ' after:' + from_

        if to_ and not when:
            to_ = self.__from_to_helper(validate=to_)
            query += ' before:' + to_

        if helper == True:
            query = self.__search_helper(query)

        search_ceid = self.__ceid()
        search_ceid = search_ceid.replace('?', '&')

        d = self.__parse_feed(self.BASE_URL + '/search?q={}'.format(query) + search_ceid, proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

In [4]:
gn=NEWS()

In [ ]:
#print(gn.top_news())

In [ ]:
#gn.topic_headlines("sports")

In [ ]:
#gn.topic_headlines(topic="sports")

In [5]:
def TOPNEWS(N):
    for i in range(0,N):
        j=i+1
        print("--------TOP NEWS %s----------"%j)
        print(gn.top_news()["entries"][i]["title"])
        print("\n")

In [6]:
TOPNEWS(12)

--------TOP NEWS 1----------
Trump attacks his own CDC scientists over how to reopen schools safely - USA TODAY


--------TOP NEWS 2----------
Juan Williams says Trump 'really came out on top' in Supreme Court ruling on tax documents - Fox News


--------TOP NEWS 3----------
Kazakhstan denies Chinese government report that country has 'unknown pneumonia' outbreak more deadly than Covid-19 - CNN


--------TOP NEWS 4----------
Mnuchin wants 'another round' of stimulus checks passed this month - New York Post


--------TOP NEWS 5----------
Who Is Keisha Lance Bottoms? Atlanta Mayor On Biden VP List - NPR


--------TOP NEWS 6----------
Goya’s CEO said the U.S. is ‘truly blessed’ with President Trump. Latinos are now boycotting. - The Washington Post


--------TOP NEWS 7----------
US Supreme Court rules half of Oklahoma is Native American land - BBC News


--------TOP NEWS 8----------
New audio brings detail to conflicting accounts of Breonna Taylor's killing - CNN


--------TOP NEWS 9-----

In [7]:
def HEADLINES(N,TOPIC):
      for i in range(0,N):
        j=i+1
        print("--------%s NEWS %s----------"%(TOPIC,j))
        print(gn.topic_headlines(topic=TOPIC)["entries"][i]["title"])
        print("\n")

In [11]:
HEADLINES(12,"SPORTS")

--------SPORTS NEWS 1----------
Richard Sherman, other NFL players rip league health guidelines - Fox News


--------SPORTS NEWS 2----------
WATCH: UFC fighter Mike Perry uses racial slurs, throws punches at Lubbock, Texas, restaurant - KCBD


--------SPORTS NEWS 3----------
Reigning champions Seattle Sounders ready for teams' maximum effort at MLS is Back Tournament - MLSsoccer.com


--------SPORTS NEWS 4----------
Big Ten Statement on 2020-21 Fall Season - Big Ten Conference


--------SPORTS NEWS 5----------
Champions League DREAM draws! Which bracket provides the juiciest matchups? | ESPN FC - ESPN UK


--------SPORTS NEWS 6----------
Smoot on Darrell Green: 'Darrell will always be cornerback royalty' - ProBasketballTalk


--------SPORTS NEWS 7----------
MLB releases 2021 schedule: What to know about 162-game slate, including April 1 Opening Day - CBS Sports


--------SPORTS NEWS 8----------
Jewish NFL player reacts to anti-semitic posts by Desean Jackson - CNN


--------SPORTS NEWS

In [8]:
def LOCATIONNEWS(N,LOCATION):
    for i in range(0,N):
        j=i+1
        print("--------%s NEWS %s----------"%(LOCATION,j))
        print(gn.geo_headlines(geo=LOCATION)["entries"][i]["title"])
        print("\n")

In [9]:
LOCATIONNEWS(10,"INDIA")

--------INDIA NEWS 1----------
Vikas Dubey: India police murder suspect shot dead after arrest - BBC News


--------INDIA NEWS 2----------
What Beijing Lost With Its Border Clash Blunder - The Wall Street Journal


--------INDIA NEWS 3----------
Coronavirus: Is India the next global hotspot? - BBC News


--------INDIA NEWS 4----------
These senior Indian couples found love again. Not everyone is happy about it - CNN


--------INDIA NEWS 5----------
New US visa rule leaves Indian, Chinese students in panic - Al Jazeera English


--------INDIA NEWS 6----------
India gold smuggling case sparks political row - BBC News


--------INDIA NEWS 7----------
Groom Dead, at Least 100 Sick With Coronavirus After Indian Wedding - The New York Times


--------INDIA NEWS 8----------
Pakistan invites India to file review against spy's conviction - Al Jazeera English


--------INDIA NEWS 9----------
India Surpasses Russia In Reported Coronavirus Infections : Coronavirus Live Updates - NPR


--------INDI

In [10]:
def QUERY(N,QUERY):
      for i in range(0,N):
        j=i+1
        print("--------%s :NEWS RESULT %s----------"%(QUERY,j))
        print(gn.search(query=QUERY)["entries"][i]["title"])
        print("\n")

In [11]:
QUERY(5,"WHERE IS INDIA IN CORONA")

--------WHERE IS INDIA IN CORONA :NEWS RESULT 1----------
Coronavirus: Is India the next global hotspot? - BBC News


--------WHERE IS INDIA IN CORONA :NEWS RESULT 2----------
Coronavirus latest: US, India see new daily infection highs - DW (English)


--------WHERE IS INDIA IN CORONA :NEWS RESULT 3----------
Coronavirus live updates: Around 2.7 lakh tests being done daily, Harsh Vardhan says - Times of India


--------WHERE IS INDIA IN CORONA :NEWS RESULT 4----------
India Surpasses Russia In Reported Coronavirus Infections : Coronavirus Live Updates - NPR


--------WHERE IS INDIA IN CORONA :NEWS RESULT 5----------
Record single-day jump of 26,506 Covid-19 cases in India - Times of India


